# Section2：LSTM

### RNNの課題
- 時系列を遡れば遡るほど、勾配が消失していく。　⇒　長い時系列の学習が困難。


- 解決策
前回の授業で触れた勾配消失の解決方法とは別で、構造自体を変えて解決したものがLSTM。

勾配爆発とは？
勾配が、層を逆伝播するごとに指数関数的に大きくなっていく。

## 2.1 CEC
勾配消失および勾配爆発の解決方法として、勾配が、1であれば解決できる。

- CECの課題 \
⇒　入力データについて、時間依存度に関係なく重みが一律である。 \
　⇒　ニューラルネットワークの学習特性が無いということ。

## 2.2 入力ゲートと出力ゲート

## 2.3 忘却ゲート

## 2.4 覗き穴結合

# 確認テスト

#### P62
問） \
シグモイド関数を微分した時、入力値が0の時に最大値をとる。その値として正しいものを選択肢から選べ。
1. 0.15
1. 0.25
1. 0.35
1. 0.45

答） \
（２）0.25